In [13]:
import numpy as np 
import pandas as pd 

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier 

from sklearn.metrics import roc_auc_score

In [18]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [19]:
test_id = test_df['ID_code']
train_df.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [4]:
print(train_df['target'].value_counts())
unsatisfied_cnt = train_df[train_df['target'] == 1].target.count()
total_cnt = train_df.target.count()
print('unsatisfied 비율은 {0:.2f}'.format((unsatisfied_cnt / total_cnt)))

0    179902
1     20098
Name: target, dtype: int64
unsatisfied 비율은 0.10


In [5]:
train_df.describe( )

,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,...,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,0.100490,10.679914,-1.627622,10.715192,6.796529,11.078333,-5.065317,5.408949,16.545850,0.284162,...,3.234440,7.438408,1.927839,3.331774,17.993784,-0.142088,2.303335,8.908158,15.870720,-3.326537
std,0.300653,3.040051,4.050044,2.640894,2.043319,1.623150,7.863267,0.866607,3.418076,3.332634,...,4.559922,3.023272,1.478423,3.992030,3.135162,1.429372,5.454369,0.921625,3.010945,10.438015
min,0.000000,0.408400,-15.043400,2.117100,-0.040200,5.074800,-32.562600,2.347300,5.349700,-10.505500,...,-14.093300,-2.691700,-3.814500,-11.783400,8.694400,-5.261000,-14.209600,5.960600,6.299300,-38.852800
25%,0.000000,8.453850,-4.740025,8.722475,5.254075,9.883175,-11.200350,4.767700,13.943800,-2.317800,...,-0.058825,5.157400,0.889775,0.584600,15.629800,-1.170700,-1.946925,8.252800,13.829700,-11.208475
50%,0.000000,10.524750,-1.608050,10.580000,6.825000,11.108250,-4.833150,5.385100,16.456800,0.393700,...,3.203600,7.347750,1.901300,3.396350,17.957950,-0.172700,2.408900,8.888200,15.934050,-2.819550
75%,0.000000,12.758200,1.358625,12.516700,8.324100,12.261125,0.924800,6.003000,19.102900,2.937900,...,6.406200,9.512525,2.949500,6.205800,20.396525,0.829600,6.556725,9.593300,18.064725,4.836800
max,1.000000,20.315000,10.376800,19.353000,13.188300,16.671400,17.251600,8.447700,27.691800,10.151300,...,18.440900,16.716500,8.402400,18.281800,27.928800,4.272900,18.321500,12.000400,26.079100,28.500700


In [6]:
#object=id, int=target
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Columns: 202 entries, ID_code to var_199
dtypes: float64(200), int64(1), object(1)
memory usage: 308.2+ MB


In [7]:
# var3 피처 값 대체 및 ID 피처 드롭
train_df.drop('ID_code',axis=1 , inplace=True)
test_df.drop('ID_code',axis=1 , inplace=True)

In [8]:
# 피처 세트와 레이블 세트분리. 레이블 컬럼은 DataFrame의 맨 마지막에 위치해 컬럼 위치 -1로 분리
X_features = train_df.iloc[:, 1:]
y_labels = train_df.iloc[:, -0]
print('피처 데이터 shape:{0}'.format(X_features.shape))

피처 데이터 shape:(200000, 200)


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_features, y_labels,
                                                    test_size=0.2, random_state=0)
train_cnt = y_train.count()
test_cnt = y_test.count()
print('학습 세트 Shape:{0}, 테스트 세트 Shape:{1}'.format(X_train.shape , X_test.shape))

print(' 학습 세트 레이블 값 분포 비율')
print(y_train.value_counts()/train_cnt)
print('\n 테스트 세트 레이블 값 분포 비율')
print(y_test.value_counts()/test_cnt)

학습 세트 Shape:(160000, 200), 테스트 세트 Shape:(40000, 200)
 학습 세트 레이블 값 분포 비율
0    0.899475
1    0.100525
Name: target, dtype: float64

 테스트 세트 레이블 값 분포 비율
0    0.89965
1    0.10035
Name: target, dtype: float64


#  LGBoost

In [15]:
lgbm_clf = LGBMClassifier(    
    n_estimators=10000,
    bagging_fraction= 0.3,
    boost_from_average= False,
    boost= 'gbdt',
    feature_fraction= 0.041,
    learning_rate= 0.03,
    max_depth= -1,
    metric='aud',
    min_data_in_leaf= 80,
    min_sum_hessian_in_leaf= 10.0,
    num_leaves= 13,
    num_threads= 8,
    tree_learner= 'serial',
    objective= 'binary', 
    verbosity= -1
)

evals = [(X_test, y_test)]
lgbm_clf.fit(X_train, y_train, early_stopping_rounds=100, eval_metric="auc", eval_set=evals,
                verbose=True)

lgbm_roc_score = roc_auc_score(y_test, lgbm_clf.predict_proba(X_test)[:,1],average='macro')
print('ROC AUC: {0:.4f}'.format(lgbm_roc_score))

[1]	valid_0's auc: 0.577134
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's auc: 0.619391
[3]	valid_0's auc: 0.65068
[4]	valid_0's auc: 0.670313
[5]	valid_0's auc: 0.674364
[6]	valid_0's auc: 0.690533
[7]	valid_0's auc: 0.709807
[8]	valid_0's auc: 0.712528
[9]	valid_0's auc: 0.71798
[10]	valid_0's auc: 0.719577
[11]	valid_0's auc: 0.727271
[12]	valid_0's auc: 0.728994
[13]	valid_0's auc: 0.738302
[14]	valid_0's auc: 0.744055
[15]	valid_0's auc: 0.752205
[16]	valid_0's auc: 0.762391
[17]	valid_0's auc: 0.768569
[18]	valid_0's auc: 0.773034
[19]	valid_0's auc: 0.774185
[20]	valid_0's auc: 0.770691
[21]	valid_0's auc: 0.769324
[22]	valid_0's auc: 0.780172
[23]	valid_0's auc: 0.781409
[24]	valid_0's auc: 0.778917
[25]	valid_0's auc: 0.783836
[26]	valid_0's auc: 0.787566
[27]	valid_0's auc: 0.786374
[28]	valid_0's auc: 0.784972
[29]	valid_0's auc: 0.787993
[30]	valid_0's auc: 0.79325
[31]	valid_0's auc: 0.794382
[32]	valid_0's auc: 0.797654
[33]	valid_0's auc: 0

[278]	valid_0's auc: 0.872613
[279]	valid_0's auc: 0.872337
[280]	valid_0's auc: 0.872461
[281]	valid_0's auc: 0.872365
[282]	valid_0's auc: 0.872192
[283]	valid_0's auc: 0.872228
[284]	valid_0's auc: 0.872181
[285]	valid_0's auc: 0.872137
[286]	valid_0's auc: 0.872383
[287]	valid_0's auc: 0.872548
[288]	valid_0's auc: 0.872615
[289]	valid_0's auc: 0.872656
[290]	valid_0's auc: 0.87296
[291]	valid_0's auc: 0.872976
[292]	valid_0's auc: 0.872895
[293]	valid_0's auc: 0.87286
[294]	valid_0's auc: 0.872861
[295]	valid_0's auc: 0.872855
[296]	valid_0's auc: 0.872867
[297]	valid_0's auc: 0.872982
[298]	valid_0's auc: 0.873045
[299]	valid_0's auc: 0.873129
[300]	valid_0's auc: 0.873225
[301]	valid_0's auc: 0.873225
[302]	valid_0's auc: 0.873182
[303]	valid_0's auc: 0.873233
[304]	valid_0's auc: 0.873334
[305]	valid_0's auc: 0.873492
[306]	valid_0's auc: 0.873687
[307]	valid_0's auc: 0.873732
[308]	valid_0's auc: 0.87367
[309]	valid_0's auc: 0.873816
[310]	valid_0's auc: 0.873937
[311]	valid_0

[555]	valid_0's auc: 0.882792
[556]	valid_0's auc: 0.882808
[557]	valid_0's auc: 0.882861
[558]	valid_0's auc: 0.882893
[559]	valid_0's auc: 0.882899
[560]	valid_0's auc: 0.882954
[561]	valid_0's auc: 0.882986
[562]	valid_0's auc: 0.883013
[563]	valid_0's auc: 0.883023
[564]	valid_0's auc: 0.883065
[565]	valid_0's auc: 0.883066
[566]	valid_0's auc: 0.8831
[567]	valid_0's auc: 0.883101
[568]	valid_0's auc: 0.883106
[569]	valid_0's auc: 0.883096
[570]	valid_0's auc: 0.883082
[571]	valid_0's auc: 0.883048
[572]	valid_0's auc: 0.883095
[573]	valid_0's auc: 0.883075
[574]	valid_0's auc: 0.883123
[575]	valid_0's auc: 0.883178
[576]	valid_0's auc: 0.883193
[577]	valid_0's auc: 0.883201
[578]	valid_0's auc: 0.883215
[579]	valid_0's auc: 0.88322
[580]	valid_0's auc: 0.883277
[581]	valid_0's auc: 0.883287
[582]	valid_0's auc: 0.883274
[583]	valid_0's auc: 0.883258
[584]	valid_0's auc: 0.88333
[585]	valid_0's auc: 0.883243
[586]	valid_0's auc: 0.883208
[587]	valid_0's auc: 0.883288
[588]	valid_0'

[831]	valid_0's auc: 0.887715
[832]	valid_0's auc: 0.887729
[833]	valid_0's auc: 0.887738
[834]	valid_0's auc: 0.88778
[835]	valid_0's auc: 0.887798
[836]	valid_0's auc: 0.887782
[837]	valid_0's auc: 0.887825
[838]	valid_0's auc: 0.887831
[839]	valid_0's auc: 0.88783
[840]	valid_0's auc: 0.887853
[841]	valid_0's auc: 0.887869
[842]	valid_0's auc: 0.887908
[843]	valid_0's auc: 0.887926
[844]	valid_0's auc: 0.887922
[845]	valid_0's auc: 0.88793
[846]	valid_0's auc: 0.887956
[847]	valid_0's auc: 0.887933
[848]	valid_0's auc: 0.887948
[849]	valid_0's auc: 0.887958
[850]	valid_0's auc: 0.887989
[851]	valid_0's auc: 0.888017
[852]	valid_0's auc: 0.887997
[853]	valid_0's auc: 0.887986
[854]	valid_0's auc: 0.888017
[855]	valid_0's auc: 0.888047
[856]	valid_0's auc: 0.888032
[857]	valid_0's auc: 0.88803
[858]	valid_0's auc: 0.888048
[859]	valid_0's auc: 0.888076
[860]	valid_0's auc: 0.888083
[861]	valid_0's auc: 0.888126
[862]	valid_0's auc: 0.888151
[863]	valid_0's auc: 0.88818
[864]	valid_0's

[1102]	valid_0's auc: 0.890798
[1103]	valid_0's auc: 0.890818
[1104]	valid_0's auc: 0.890819
[1105]	valid_0's auc: 0.890841
[1106]	valid_0's auc: 0.890851
[1107]	valid_0's auc: 0.890877
[1108]	valid_0's auc: 0.890907
[1109]	valid_0's auc: 0.890913
[1110]	valid_0's auc: 0.890919
[1111]	valid_0's auc: 0.890938
[1112]	valid_0's auc: 0.890952
[1113]	valid_0's auc: 0.890963
[1114]	valid_0's auc: 0.890979
[1115]	valid_0's auc: 0.890985
[1116]	valid_0's auc: 0.890992
[1117]	valid_0's auc: 0.891021
[1118]	valid_0's auc: 0.891019
[1119]	valid_0's auc: 0.891026
[1120]	valid_0's auc: 0.891008
[1121]	valid_0's auc: 0.891021
[1122]	valid_0's auc: 0.89104
[1123]	valid_0's auc: 0.891077
[1124]	valid_0's auc: 0.891074
[1125]	valid_0's auc: 0.891087
[1126]	valid_0's auc: 0.891097
[1127]	valid_0's auc: 0.891094
[1128]	valid_0's auc: 0.891116
[1129]	valid_0's auc: 0.891112
[1130]	valid_0's auc: 0.891114
[1131]	valid_0's auc: 0.891119
[1132]	valid_0's auc: 0.891114
[1133]	valid_0's auc: 0.891149
[1134]	va

...

[1635]	valid_0's auc: 0.894057
[1636]	valid_0's auc: 0.894061
[1637]	valid_0's auc: 0.894071
[1638]	valid_0's auc: 0.894063
[1639]	valid_0's auc: 0.894054
[1640]	valid_0's auc: 0.894052
[1641]	valid_0's auc: 0.894059
[1642]	valid_0's auc: 0.894056
[1643]	valid_0's auc: 0.894051
[1644]	valid_0's auc: 0.894053
[1645]	valid_0's auc: 0.894054
[1646]	valid_0's auc: 0.89407
[1647]	valid_0's auc: 0.894074
[1648]	valid_0's auc: 0.894087
[1649]	valid_0's auc: 0.894091
[1650]	valid_0's auc: 0.894104
[1651]	valid_0's auc: 0.894107
[1652]	valid_0's auc: 0.894114
[1653]	valid_0's auc: 0.894112
[1654]	valid_0's auc: 0.894099
[1655]	valid_0's auc: 0.894101
[1656]	valid_0's auc: 0.894095
[1657]	valid_0's auc: 0.894103
[1658]	valid_0's auc: 0.894103
[1659]	valid_0's auc: 0.894098
[1660]	valid_0's auc: 0.89409
[1661]	valid_0's auc: 0.894107
[1662]	valid_0's auc: 0.894114
[1663]	valid_0's auc: 0.89411
[1664]	valid_0's auc: 0.894134
[1665]	valid_0's auc: 0.894128
[1666]	valid_0's auc: 0.89414
[1667]	valid

[1901]	valid_0's auc: 0.895065
[1902]	valid_0's auc: 0.895063
[1903]	valid_0's auc: 0.895066
[1904]	valid_0's auc: 0.895061
[1905]	valid_0's auc: 0.895056
[1906]	valid_0's auc: 0.895047
[1907]	valid_0's auc: 0.895058
[1908]	valid_0's auc: 0.895076
[1909]	valid_0's auc: 0.895082
[1910]	valid_0's auc: 0.895095
[1911]	valid_0's auc: 0.8951
[1912]	valid_0's auc: 0.895107
[1913]	valid_0's auc: 0.895115
[1914]	valid_0's auc: 0.895121
[1915]	valid_0's auc: 0.895134
[1916]	valid_0's auc: 0.895127
[1917]	valid_0's auc: 0.895116
[1918]	valid_0's auc: 0.89511
[1919]	valid_0's auc: 0.895116
[1920]	valid_0's auc: 0.89511
[1921]	valid_0's auc: 0.895108
[1922]	valid_0's auc: 0.895109
[1923]	valid_0's auc: 0.895103
[1924]	valid_0's auc: 0.89511
[1925]	valid_0's auc: 0.89512
[1926]	valid_0's auc: 0.89513
[1927]	valid_0's auc: 0.895151
[1928]	valid_0's auc: 0.895163
[1929]	valid_0's auc: 0.895176
[1930]	valid_0's auc: 0.895183
[1931]	valid_0's auc: 0.895183
[1932]	valid_0's auc: 0.895186
[1933]	valid_0'

[2167]	valid_0's auc: 0.895545
[2168]	valid_0's auc: 0.895543
[2169]	valid_0's auc: 0.895545
[2170]	valid_0's auc: 0.895547
[2171]	valid_0's auc: 0.895556
[2172]	valid_0's auc: 0.895557
[2173]	valid_0's auc: 0.895563
[2174]	valid_0's auc: 0.895563
[2175]	valid_0's auc: 0.895567
[2176]	valid_0's auc: 0.895547
[2177]	valid_0's auc: 0.89555
[2178]	valid_0's auc: 0.895555
[2179]	valid_0's auc: 0.89556
[2180]	valid_0's auc: 0.895569
[2181]	valid_0's auc: 0.89556
[2182]	valid_0's auc: 0.895557
[2183]	valid_0's auc: 0.895557
[2184]	valid_0's auc: 0.895548
[2185]	valid_0's auc: 0.89555
[2186]	valid_0's auc: 0.89556
[2187]	valid_0's auc: 0.895562
[2188]	valid_0's auc: 0.895581
[2189]	valid_0's auc: 0.895586
[2190]	valid_0's auc: 0.895584
[2191]	valid_0's auc: 0.895597
[2192]	valid_0's auc: 0.895598
[2193]	valid_0's auc: 0.895589
[2194]	valid_0's auc: 0.895579
[2195]	valid_0's auc: 0.89557
[2196]	valid_0's auc: 0.895571
[2197]	valid_0's auc: 0.895573
[2198]	valid_0's auc: 0.895581
[2199]	valid_0

[2434]	valid_0's auc: 0.895929
[2435]	valid_0's auc: 0.895928
[2436]	valid_0's auc: 0.895941
[2437]	valid_0's auc: 0.895947
[2438]	valid_0's auc: 0.895956
[2439]	valid_0's auc: 0.895948
[2440]	valid_0's auc: 0.895942
[2441]	valid_0's auc: 0.895958
[2442]	valid_0's auc: 0.895963
[2443]	valid_0's auc: 0.895959
[2444]	valid_0's auc: 0.895969
[2445]	valid_0's auc: 0.89597
[2446]	valid_0's auc: 0.895966
[2447]	valid_0's auc: 0.895958
[2448]	valid_0's auc: 0.895971
[2449]	valid_0's auc: 0.895974
[2450]	valid_0's auc: 0.895978
[2451]	valid_0's auc: 0.895985
[2452]	valid_0's auc: 0.895988
[2453]	valid_0's auc: 0.89599
[2454]	valid_0's auc: 0.895986
[2455]	valid_0's auc: 0.89599
[2456]	valid_0's auc: 0.896001
[2457]	valid_0's auc: 0.895992
[2458]	valid_0's auc: 0.895995
[2459]	valid_0's auc: 0.895998
[2460]	valid_0's auc: 0.895998
[2461]	valid_0's auc: 0.896002
[2462]	valid_0's auc: 0.896006
[2463]	valid_0's auc: 0.896012
[2464]	valid_0's auc: 0.896002
[2465]	valid_0's auc: 0.895987
[2466]	vali

[2702]	valid_0's auc: 0.895994
[2703]	valid_0's auc: 0.895996
[2704]	valid_0's auc: 0.895992
[2705]	valid_0's auc: 0.895981
[2706]	valid_0's auc: 0.895965
[2707]	valid_0's auc: 0.895971
[2708]	valid_0's auc: 0.895969
[2709]	valid_0's auc: 0.895975
[2710]	valid_0's auc: 0.895985
[2711]	valid_0's auc: 0.895979
Early stopping, best iteration is:
[2611]	valid_0's auc: 0.896069
ROC AUC: 0.8961


In [29]:
y_test_pred=lgbm_clf.predict_proba(test_df.iloc[:,1:])[:,1]

In [30]:
##submission
sub_df1 = pd.DataFrame({"ID_code":test_id})
sub_df1["target"] = y_test_pred
sub_df1.to_csv("lgboost.csv", index=False)